In [1]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'dataset_v1.1.zip'
    path_to_file = '/content/gdrive/MyDrive/Processed Data/dataset/dataset_v1.1/'
    print(path_to_file)

py_file_location = "/content/gdrive/My Drive/Colab Notebooks"
sys.path.append(os.path.abspath(py_file_location))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Processed Data/dataset/dataset_v1.1/


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import time
import utils
import os
import pandas as pd

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


In [4]:
dataset_folder = path_to_file
#train_folder = f'{path_to_file}/train/'
#test_folder = f'{path_to_file}/test/'

##train_size = len(os.listdir(train_folder))
#test_size = len(os.listdir(test_folder))
#os.remove('.DS_Store')
#train_size,test_size
#os.listdir(test_folder)

In [5]:
#path_to_file = '/content/gdrive/My Drive/CS5242 Project/Processed Data/dataset/dataset_v1.1'
train_data=torch.load(path_to_file+'train.pt')
#train_data=torch.load(dataset_folder+'/train.pt')
#print(train_image[0])
print(train_data.size())

test_data=torch.load(path_to_file+'test.pt')
#print(train_image)
print(test_data.size())

train_label=torch.load(path_to_file+'train_label.pt')
#print(train_label)

test_label=torch.load(path_to_file+'test_label.pt')
#print(test_label)

torch.Size([980, 3, 128, 128])
torch.Size([250, 3, 128, 128])


In [53]:
class VGG_convnet(nn.Module):

    def __init__(self):

        super(VGG_convnet, self).__init__()

        # block 1:         3 x 128 x 128 --> 64 x 32 x 32        
        self.conv1a = nn.Conv2d(3,   64,  kernel_size=32, padding=16 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=32, padding=16 )
        self.pool1  = nn.MaxPool2d(4,4)

        # block 2:         64 x 32 x 32 --> 128 x 16 x 16
        self.conv2a = nn.Conv2d(64,  128, kernel_size=8, padding=4 )
        self.conv2b = nn.Conv2d(128, 128, kernel_size=8, padding=4 )
        self.pool2  = nn.MaxPool2d(2,2)

        # block 3:         128 x 16 x 16 --> 256 x 8 x 8        
        self.conv3a = nn.Conv2d(128, 256, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2,2)
        
        #block 4:          256 x 8 x 8 --> 512 x 4 x 4
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2,2)

        # linear layers:   512 x 4 x 4 --> 2048 --> 4096 --> 4096 --> 10
        self.linear1 = nn.Linear(8192, 16384)
        self.linear2 = nn.Linear(16384,16384)
        self.linear3 = nn.Linear(16384, 12)


    def forward(self, x):

        # block 1:         3 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = torch.relu(x)
        x = self.conv1b(x)
        x = torch.relu(x)
        x = self.pool1(x)

        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        x = self.conv2a(x)
        x = torch.relu(x)
        x = self.conv2b(x)
        x = torch.relu(x)
        x = self.pool2(x)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4
        x = self.conv3a(x)
        x = torch.relu(x)
        x = self.conv3b(x)
        x = torch.relu(x)
        x = self.pool3(x)

        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        x = self.conv4a(x)
        x = torch.relu(x)
        x = self.pool4(x)

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        x = x.view(-1, 8192)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.linear3(x) 
        
        return x

In [54]:
mean= train_data.mean()

print(mean)
std= train_data.std()

print(std)

tensor(0.4325)
tensor(0.2568)


In [55]:
net=VGG_convnet()

print(net)
utils.display_num_param(net)

VGG_convnet(
  (conv1a): Conv2d(3, 64, kernel_size=(32, 32), stride=(1, 1), padding=(16, 16))
  (conv1b): Conv2d(64, 64, kernel_size=(32, 32), stride=(1, 1), padding=(16, 16))
  (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2a): Conv2d(64, 128, kernel_size=(8, 8), stride=(1, 1), padding=(4, 4))
  (conv2b): Conv2d(128, 128, kernel_size=(8, 8), stride=(1, 1), padding=(4, 4))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=8192, out_features=16384, bias=True)
  (linear2):

In [56]:
net = net.to(device)

mean = mean.to(device)

std = std.to(device)

In [57]:
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 10

In [58]:
def eval_on_test_set():

    running_error=0
    num_batches=0

    for i in range(0,250,bs):

        minibatch_data =  test_data[i:i+bs]
        minibatch_label= test_label[i:i+bs]

        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        inputs = (minibatch_data - mean)/std

        scores=net( inputs ) 

        error = utils.get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1

    total_error = running_error/num_batches
    print( 'error rate on test set =', total_error*100 ,'percent')

In [ ]:
start=time.time()

for epoch in range(1,10):
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch==5 or epoch == 7 or epoch==9:
        my_lr = my_lr / 2
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    running_error=0
    num_batches=0
    
    # set the order in which to visit the image from the training set
    shuffled_indices=torch.randperm(980)
 
    for count in range(0,980,bs):
    
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch       
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices]
        minibatch_label=  train_label[indices]
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # normalize the minibatch (this is the only difference compared to before!)
        inputs = (minibatch_data - mean)/std
        
        # tell Pytorch to start tracking all operations that will be done on "inputs"
        inputs.requires_grad_()

        # forward the minibatch through the net 
        scores=net( inputs ) 

        # Compute the average of the losses of the data points in the minibatch
        loss =  criterion( scores , minibatch_label) 
        
        # backward pass to compute dL/dU, dL/dV and dL/dW   
        loss.backward()

        # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
        optimizer.step()
        

        # START COMPUTING STATS
        
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item()
        
        # compute the error made on this batch and add it to the running error       
        error = utils.get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    

    print('epoch=',epoch, '\t time=', elapsed,'min','\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    eval_on_test_set() 
    print(' ')

In [ ]:
# choose a picture at random
idx=randint(0, 249)
im=test_data[idx]

# diplay the picture
utils.show(im)
print(test_label[idx])
# send to device, rescale, and view as a batch of 1 
#im = im.to(device)
#im= (im-mean) / std
#im=im.view(1,3,128,128)

# feed it to the net and display the confidence scores
#scores =  net(im) 
##probs= torch.softmax(scores, dim=1)
#utils.show_prob_cifar(probs.cpu())